In [502]:
# /// script
# requires-python = ">=3.11"
# dependencies = [
#     "importlib",
#     "pandas",
#     "udi-grammar-py",``
# ]
# ///

In [503]:
from udi_grammar_py import Chart, Op, rolling
import pandas as pd
import sys
import template_generation
import schema_generation
import template_expansion
import paraphraser
import importlib
import json
sys.path.append('.')

## Use python to generate template training data.

In [504]:
importlib.reload(template_generation)
df = template_generation.generate()
template_question_count = df.shape[0]
df.head()

,query_template,constraints,spec_template,query_type,creation_method
0,"How many <E1> are there, grouped by <E2.F:n>?","[E2.F.c * 2 < E1.c, E2.F.c < 4, E1.r.E2.c.to =...","{""source"": [{""name"": ""<E1>"", ""source"": ""<E1.ur...",question,template
1,"How many <E1> are there, grouped by <E2.F:n>?","[E2.F.c * 2 < E1.c, E2.F.c >= 4, E2.F.c < 25, ...","{""source"": [{""name"": ""<E1>"", ""source"": ""<E1.ur...",question,template


## update data schema based on files in ./datasets folder

In [505]:
importlib.reload(schema_generation)
schema_generation.main()

## Contextualize the template training data by putting in real entity names and fields if they satisfy the constraints.

In [506]:
importlib.reload(template_expansion)
with open('./datasets/schema.json') as f:
    schema_list = json.load(f)
    df = template_expansion.expand(df, schema_list)

df.head()

⭐ constraints ⭐
["E2_F['cardinality'] * 2 < E1['cardinality']",
 "E2_F['cardinality'] < 4",
 "E2['entity'] in E1['relationships']",
 "E1['relationships'][E2['entity']]['cardinality']['to'] == 'one'",
 "E2_F['data_type'] in ['nominal']",
 "E1['entity'] not in [E2['entity']]",
 "E2['entity'] not in [E1['entity']]",
 "E2_F['entity'] == E2['entity']"]
⭐ entities ⭐
['E1', 'E2']
[{'cardinality': 3235,
  'data_type': None,
  'entity': 'datasets',
  'name': None,
  'relationships': {'donors': {'cardinality': {'from': 'many', 'to': 'one'},
                               'id': {'from': 'donor.hubmap_id',
                                      'to': 'hubmap_id'}}},
  'url': './data/hubmap/datasets.csv'},
 {'cardinality': 2354,
  'data_type': None,
  'entity': 'samples',
  'name': None,
  'relationships': {'donors': {'cardinality': {'from': 'many', 'to': 'one'},
                               'id': {'from': 'donor.hubmap_id',
                                      'to': 'hubmap_id'}}},
  'url': './d

,query_template,constraints,spec_template,query_type,creation_method,query_base,spec,solution,dataset_schema
0,"How many <E1> are there, grouped by <E2.F:n>?","[E2.F.c * 2 < E1.c, E2.F.c < 4, E1.r.E2.c.to =...","{""source"": [{""name"": ""<E1>"", ""source"": ""<E1.ur...",question,template,"How many samples are there, grouped by weight_...","{""source"": [{""name"": ""samples"", ""source"": ""./d...","{'E1': {'entity': 'samples', 'url': './data/hu...",hubmap
0,"How many <E1> are there, grouped by <E2.F:n>?","[E2.F.c * 2 < E1.c, E2.F.c < 4, E1.r.E2.c.to =...","{""source"": [{""name"": ""<E1>"", ""source"": ""<E1.ur...",question,template,"How many samples are there, grouped by sex?","{""source"": [{""name"": ""samples"", ""source"": ""./d...","{'E1': {'entity': 'samples', 'url': './data/hu...",hubmap
0,"How many <E1> are there, grouped by <E2.F:n>?","[E2.F.c * 2 < E1.c, E2.F.c < 4, E1.r.E2.c.to =...","{""source"": [{""name"": ""<E1>"", ""source"": ""<E1.ur...",question,template,"How many samples are there, grouped by serum_c...","{""source"": [{""name"": ""samples"", ""source"": ""./d...","{'E1': {'entity': 'samples', 'url': './data/hu...",hubmap
0,"How many <E1> are there, grouped by <E2.F:n>?","[E2.F.c * 2 < E1.c, E2.F.c < 4, E1.r.E2.c.to =...","{""source"": [{""name"": ""<E1>"", ""source"": ""<E1.ur...",question,template,"How many samples are there, grouped by rh_bloo...","{""source"": [{""name"": ""samples"", ""source"": ""./d...","{'E1': {'entity': 'samples', 'url': './data/hu...",hubmap
0,"How many <E1> are there, grouped by <E2.F:n>?","[E2.F.c * 2 < E1.c, E2.F.c < 4, E1.r.E2.c.to =...","{""source"": [{""name"": ""<E1>"", ""source"": ""<E1.ur...",question,template,"How many samples are there, grouped by patholo...","{""source"": [{""name"": ""samples"", ""source"": ""./d...","{'E1': {'entity': 'samples', 'url': './data/hu...",hubmap


## The paraphraser will use LLM frameork to paraphrase the query_base into several options


In [507]:
importlib.reload(paraphraser)
df = paraphraser.paraphrase(df)
df.head()


,query_template,constraints,spec_template,query_type,creation_method,query_base,spec,solution,dataset_schema,query,expertise,formality
0,"How many <E1> are there, grouped by <E2.F:n>?","[E2.F.c * 2 < E1.c, E2.F.c < 4, E1.r.E2.c.to =...","{""source"": [{""name"": ""<E1>"", ""source"": ""<E1.ur...",question,template,"How many samples are there, grouped by weight_...","{""source"": [{""name"": ""samples"", ""source"": ""./d...","{'E1': {'entity': 'samples', 'url': './data/hu...",hubmap,"How many samples are there, grouped by weight_...",-1,-1
0,"How many <E1> are there, grouped by <E2.F:n>?","[E2.F.c * 2 < E1.c, E2.F.c < 4, E1.r.E2.c.to =...","{""source"": [{""name"": ""<E1>"", ""source"": ""<E1.ur...",question,template,"How many samples are there, grouped by sex?","{""source"": [{""name"": ""samples"", ""source"": ""./d...","{'E1': {'entity': 'samples', 'url': './data/hu...",hubmap,"How many samples are there, grouped by sex?",-1,-1
0,"How many <E1> are there, grouped by <E2.F:n>?","[E2.F.c * 2 < E1.c, E2.F.c < 4, E1.r.E2.c.to =...","{""source"": [{""name"": ""<E1>"", ""source"": ""<E1.ur...",question,template,"How many samples are there, grouped by serum_c...","{""source"": [{""name"": ""samples"", ""source"": ""./d...","{'E1': {'entity': 'samples', 'url': './data/hu...",hubmap,"How many samples are there, grouped by serum_c...",-1,-1
0,"How many <E1> are there, grouped by <E2.F:n>?","[E2.F.c * 2 < E1.c, E2.F.c < 4, E1.r.E2.c.to =...","{""source"": [{""name"": ""<E1>"", ""source"": ""<E1.ur...",question,template,"How many samples are there, grouped by rh_bloo...","{""source"": [{""name"": ""samples"", ""source"": ""./d...","{'E1': {'entity': 'samples', 'url': './data/hu...",hubmap,"How many samples are there, grouped by rh_bloo...",-1,-1
0,"How many <E1> are there, grouped by <E2.F:n>?","[E2.F.c * 2 < E1.c, E2.F.c < 4, E1.r.E2.c.to =...","{""source"": [{""name"": ""<E1>"", ""source"": ""<E1.ur...",question,template,"How many samples are there, grouped by patholo...","{""source"": [{""name"": ""samples"", ""source"": ""./d...","{'E1': {'entity': 'samples', 'url': './data/hu...",hubmap,"How many samples are there, grouped by patholo...",-1,-1


## Export as json object

In [508]:
df.to_json('./out/training_data.json', orient='records')

In [509]:
print(df.columns)

Index(['query_template', 'constraints', 'spec_template', 'query_type',
       'creation_method', 'query_base', 'spec', 'solution', 'dataset_schema',
       'query', 'expertise', 'formality'],
      dtype='object')


In [510]:
# print(df.shape)
expanded_question_count = df.shape[0]
print(f"Generated {template_question_count} templates and expanded to {expanded_question_count} questions.")

Generated 2 templates and expanded to 48 questions.


In [511]:
# queries = df['query'].tolist()
# for q in queries:
#     print(q)